In [1]:
import os
import googleapiclient.discovery
import pandas as pd
import time

# API Key 설정
API_KEY = ""  # 여기에 자신의 YouTube API 키를 입력하세요.
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

# 유튜브 클라이언트 초기화
youtube = googleapiclient.discovery.build(
    YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=API_KEY
)

# Gaming 카테고리 동영상 정보 가져오기
def get_top_gaming_videos(region_code="KR", category_id="20", max_results=300):
    videos = []
    next_page_token = None

    while len(videos) < max_results:
        try:
            request = youtube.videos().list(
                part="snippet",
                chart="mostPopular",
                regionCode=region_code,
                videoCategoryId=category_id,  # Gaming 카테고리 설정
                maxResults=50,
                pageToken=next_page_token
            )
            response = request.execute()
            videos.extend([
                {
                    "video_id": item["id"],
                    "video_title": item["snippet"]["title"]
                }
                for item in response.get("items", [])
            ])
            next_page_token = response.get("nextPageToken")
            if not next_page_token:
                break
        except Exception as e:
            print(f"Gaming 동영상 가져오기 중 오류 발생: {e}")
            break

    return videos[:max_results]

# 댓글 가져오기
def get_video_comments(video_id, video_title, max_comments=10000):
    comments = []
    next_page_token = None

    while len(comments) < max_comments:
        try:
            request = youtube.commentThreads().list(
                part="snippet,replies",
                videoId=video_id,
                maxResults=100,
                pageToken=next_page_token
            )
            response = request.execute()

            for item in response.get("items", []):
                comment = item["snippet"]["topLevelComment"]["snippet"]
                comment_id = item["id"]
                parent_id = None  # Top-level comment has no parent

                # Add top-level comment
                comments.append({
                    "comment_id": comment_id,
                    "parent_id": parent_id,
                    "author": comment["authorDisplayName"],
                    "text": comment["textDisplay"],
                    "likes": comment["likeCount"],
                    "is_reply": False,
                    "video_id": video_id,
                    "video_title": video_title
                })

                # Add replies, if any
                for reply in item.get("replies", {}).get("comments", []):
                    comments.append({
                        "comment_id": reply["id"],
                        "parent_id": comment_id,
                        "author": reply["snippet"]["authorDisplayName"],
                        "text": reply["snippet"]["textDisplay"],
                        "likes": reply["snippet"]["likeCount"],
                        "is_reply": True,
                        "video_id": video_id,
                        "video_title": video_title
                    })

            next_page_token = response.get("nextPageToken")
            if not next_page_token:
                break
        except Exception as e:
            print(f"댓글 가져오기 중 오류 발생(video_id: {video_id}): {e}")
            break

    return comments[:max_comments]

# 동영상 및 댓글 저장
def analyze_and_save_comments(videos, output_dir="gaming"):
    os.makedirs(output_dir, exist_ok=True)
    all_comments = []

    for video in videos:
        video_id = video["video_id"]
        video_title = video["video_title"]

        print(f"댓글 가져오는 중: {video_title} ({video_id})")
        comments = get_video_comments(video_id, video_title)
        all_comments.extend(comments)
        time.sleep(1)  # 요청 간 대기 시간

    # Save all comments to a CSV file
    if all_comments:
        comments_df = pd.DataFrame(all_comments)
        output_path = os.path.join(output_dir, "comments_gaming.csv")
        comments_df.to_csv(output_path, index=False, encoding="utf-8-sig")
        print(f"댓글 저장 완료: {output_path}")
    else:
        print("댓글이 없습니다.")

# 전체 처리 함수
def process_top_gaming_videos_korea():
    output_dir = "gaming"

    # 동영상 정보 가져오기
    videos = get_top_gaming_videos(region_code="KR", category_id="20", max_results=300)

    # 댓글 크롤링 및 저장
    analyze_and_save_comments(videos, output_dir)

# 실행
process_top_gaming_videos_korea()

댓글 가져오는 중: 3성 장로 (rlDXtrvZAVs)
댓글 가져오는 중: 드디어 보는구나 5코 3성 (6zB8uhf0QbY)
댓글 가져오는 중: 🔥뽀삐한테 농담시키기🔥 (5Fq1U4mO9Qs)
댓글 가져오는 중: 암베사는 와인이다 (rQ4W_20D9No)
댓글 가져오는 중: 감독 코치가 전부 물소잖아 이거 억까야? 【MKSI 2024 EP.5】 (V2K7z-uOVg0)
댓글 가져오는 중: 지옥에서 살아 돌아왔다 (wmw1XY_xxUA)
댓글 가져오는 중: 한명씩 죽어가는 친구들...친구인척 하는 살인마를 찾아라!!! (로블록스 머더) (taw4aTO9G_s)
댓글 가져오는 중: 제발 꺼져 따라오지마 나 1등해야도ㄴㅒㅇㅗㅓ려ㅣㅍㅊㅕ (6WJ3cvjWKXE)
댓글 가져오는 중: 우주 수준의 싸움 (W7UWmKD6Iu8)
댓글 가져오는 중: 장원영 닮은 리신장인 키우기ㄷㄷ 【 MKSI 김민교팀 】 (DkM-TqiZXco)
댓글 가져오는 중: 어쩌다보니 특수 블럭 텔레파시 (a00rRLObNA8)
댓글 가져오는 중: 액체가 되어 어떤 곳이든 잠입할 수 있는 잉크 인간 (Ufo8AfYy6hg)
댓글 가져오는 중: 돌아온 유튜버들을 미치게 하는 게임 1위 (dlXQe6-izJU)
댓글 가져오는 중: MKSI 최약체 팀? 저감독이 해내보겠습니다.. (rfQAW9hSu7k)
댓글 가져오는 중: 3-2부터 키워서 모든 업그레이드를 다 한 궁극의 럼블 (4F5flvT7Xjw)
댓글 가져오는 중: 제 옛날 컨텐츠들을 구현했다고요? ㅋㅋㅋㅋㅋ(마인크래프트) (SG4K5Vi0wJU)
댓글 가져오는 중: 머리 위 수상한 숫자? (BKHtpXE1kVM)
댓글 가져오는 중: 얼굴이 바뀌는 어플..? 근데 에러가 발생했어요! (GFr1TSJinmo)
댓글 가져오는 중: 2-1에 화공 찾자고 리롤 하는 사람 (Vcqwx5wphiE)
댓글 가져오는 중: 여자 스트리머 4명과 합숙 시작 【24 MKSI EP.01】 (DER8zbHrml0)
댓글 가져오는 중: 어 너랑 리게임 안함, 당장 꺼져!

In [15]:
import os
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 스팸 패턴 정의
SPAM_PATTERNS = [
    r"(클릭|이름|프로필|보러가기|링크|구독|프사|채널)",
    r"9금",  # "I9" 또는 "19" 관련
]

def is_korean(text):
    """
    텍스트에 한글이 포함되어 있는지 확인
    """
    return bool(re.search(r"[가-힣]", text))

def find_similar_comments(comments, threshold=0.8):
    """
    댓글 간 유사도 분석
    """
    if len(comments) < 2:
        return []

    vectorizer = TfidfVectorizer(stop_words="english")
    tfidf_matrix = vectorizer.fit_transform(comments)
    similarity_matrix = cosine_similarity(tfidf_matrix)

    similar_pairs = []
    n_comments = len(comments)

    for i in range(n_comments):
        for j in range(i + 1, n_comments):
            if similarity_matrix[i, j] >= threshold:
                similar_pairs.append((i, j))
    return similar_pairs

def detect_spam(comments_df):
    """
    스팸 댓글 탐지
    """
    # 대댓글 개수 계산
    comments_df["reply_count"] = comments_df.groupby("parent_id")["parent_id"].transform("count")
    comments_df["reply_count"] = comments_df["reply_count"].fillna(0)

    # 유사 댓글 및 스팸 패턴 분석
    similar_comments = find_similar_comments(comments_df["text"].astype(str).tolist())
    spam_indices = set()

    for i, j in similar_comments:
        if comments_df.iloc[j]["is_reply"]:
            comment_text = comments_df.iloc[j]["text"]
            if is_korean(comment_text) and comments_df.iloc[j]["reply_count"] >= 2:
                for pattern in SPAM_PATTERNS:
                    if re.search(pattern, comment_text):
                        spam_indices.add(j)

    comments_df["is_spam"] = comments_df.index.isin(spam_indices).astype(int)
    return comments_df

def classify_and_save_spam(input_file="gaming/comments_gaming.csv", output_file="gaming/comments_gaming_spam.csv"):
    """
    저장된 단일 댓글 데이터를 스팸 여부로 분류하여 저장
    """
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    try:
        comments_df = pd.read_csv(input_file)

        if not comments_df.empty:
            # 스팸 탐지
            classified_df = detect_spam(comments_df)
            
            # 스팸만 필터링
            spam_only_df = classified_df[classified_df["is_spam"] == 1]
            
            # 스팸 댓글 저장
            spam_only_df.to_csv(output_file, index=False, encoding="utf-8-sig")
            print(f"스팸 댓글만 저장 완료: {output_file}")
        else:
            print(f"빈 파일입니다: {input_file}")
    except Exception as e:
        print(f"파일 처리 중 오류 발생: {input_file} - {e}")

# 실행
classify_and_save_spam()


스팸 댓글만 저장 완료: youtube_data_kr/gaming/comments_spam.csv
